# one-off preparation and download of the database (optional)

It is recommended to work on a local lamin database in many instances:

1. you might want to do some dataset level preprocessing (PCA, neighboors, clustering, re-annotation, harmonization of labels, etc..) 
2. to make the loading time much faster when you do large training runs
3. to work on compute nodes that might not have access to internet

In [41]:
# initialize a local lamin database
!lamin init --storage ~/scdataloader --schema bionty

2023-12-13 16:40:32,919:INFO - HTTP Request: GET https://hub.lamin.ai/rest/v1/instance?select=%2A%2C%20account%21inner%21fk_instance_account_id_account%28%2A%29&account.handle=eq.jkobject&name=eq.scprint "HTTP/1.1 401 Unauthorized"
2023-12-13 16:40:33,431:INFO - HTTP Request: POST https://hub.lamin.ai/auth/v1/token?grant_type=password "HTTP/1.1 200 OK"
2023-12-13 16:40:33,745:INFO - HTTP Request: POST https://hub.lamin.ai/auth/v1/logout "HTTP/1.1 204 No Content"
2023-12-13 16:40:34,089:INFO - HTTP Request: GET https://hub.lamin.ai/rest/v1/instance?select=%2A%2C%20account%21inner%21fk_instance_account_id_account%28%2A%29&account.handle=eq.jkobject&name=eq.scprint "HTTP/1.1 200 OK"
2023-12-13 16:40:34,367:INFO - HTTP Request: GET https://hub.lamin.ai/rest/v1/account?select=%2A&handle=eq.jkobject "HTTP/1.1 200 OK"
2023-12-13 16:40:34,487:INFO - HTTP Request: GET https://hub.lamin.ai/rest/v1/instance?select=%2A&account_id=eq.8292b4ae-8139-4cff-ade2-fd6447734471&name=eq.scprint "HTTP/1.1 20

In [1]:
from scdataloader import utils

import lamindb as ln
import lnschema_bionty as lb
import pandas as pd

lb.settings.organism = "human"

%load_ext autoreload
%autoreload 2

💡 lamindb instance: jkobject/scprint


## load some known ontology names

first if you use a local instance you will need to populate your ontologies. 

one could just add everything by keeping the default `None` value for the `ontology` argument, but this will take a long time.

Instead, we will load only the ontologies we need. By using all the used/existing cellxgene ontology names.

In [ ]:
#you can also load it back
mydataset = ln.Dataset.filter(name="cellxgene-local").one()

In [ ]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version = "latest")
val_to_get = ['self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id']
df = census["census_data"]["homo_sapiens"].obs.read(column_names=val_to_get, value_filter="is_primary_data == True").concat().to_pandas()
df2 = census["census_data"]["mus_musculus"].obs.read(column_names=val_to_get, value_filter="is_primary_data == True").concat().to_pandas()
df.shape

(39055600, 6)

In [ ]:
utils.populate_my_ontology(lb=lb,
    organisms=["NCBITaxon:10090", "NCBITaxon:9606"],
    sex=["PATO:0000384", "PATO:0000383"],
    ethnicities=df['self_reported_ethnicity_ontology_term_id'].unique().tolist(),
    assays=list(set(df['assay_ontology_term_id'].unique()).union(df2['assay_ontology_term_id'].unique())) + ['EFO:0010961'],
    tissues=list(set(df['tissue_ontology_term_id'].unique()).union(df2['tissue_ontology_term_id'].unique())),
    # we load all possible diseases. makes it easier
    #diseases=list(set(df['disease_ontology_term_id'].unique()).union(df2['disease_ontology_term_id'].unique())),
    dev_stages=list(df['development_stage_ontology_term_id'].unique()),)

❗ now recursing through parents: this only happens once, but is much slower than bulk saving
❗ now recursing through parents: this only happens once, but is much slower than bulk saving


## OPTIONAL: add some missing ontology names

in some instances you would have to add missing ontologies in the case where you haven't loaded everything

In [ ]:
additional_tissues = {
    "UBERON:0037144": "wall of heart",
    "UBERON:0003929": "digestive tract epithelium",
    "UBERON:0002020": "gray matter",
    "UBERON:0000200": "gyrus",
    "UBERON:0000101": "lobe of lung",
    "UBERON:0001981": "blood vessel",
    "UBERON:0001474": "bone element",
}

additional_diseases = {
    "MONDO:0001106": "kidney failure",
    "MONDO:0021166": "inflammatory disease",
    "MONDO:0004992": "cancer",
    "MONDO:0004994": "cardiomyopathy",
    "MONDO:0700065": "trisomy",
    "MONDO:0021042": "glioma",
    "MONDO:0005265": "inflammatory bowel disease",
    "MONDO:0005550": "infectious disease",
    "MONDO:0005059": "leukemia",
}

additional_assays = {
    "EFO:0010184": "Smart-like",
}


Did it using the code below to figure out things we might want to add etc..

In [ ]:
mapping, anc, leafs = utils.get_ancestry_mapping(df['tissue_ontology_term_id'].unique(), lb.Tissue.filter().df(include=["parents__ontology_id"]).set_index("ontology_id"))
# getting only the leaves for which we don't have a parent
leafs = list(leafs - set.union(*[mapping[val] for val in mapping.keys()]))

In [ ]:
lb.Tissue.search(list(leafs)[108], field="ontology_id",return_queryset=True).first().view_parents()

In [ ]:
bionty_source_ds_mouse = lb.BiontySource.filter(entity="DevelopmentalStage", organism="mouse").one()
records = lb.DevelopmentalStage.from_values(df2['development_stage_ontology_term_id'].unique().tolist(), field=lb.DevelopmentalStage.ontology_id, bionty_source=bionty_source_ds_mouse)
ln.save(records)

❗ now recursing through parents: this only happens once, but is much slower than bulk saving


In [ ]:
assay = ['EFO:0010961']
records = lb.ExperimentalFactor.from_values(assay, field=lb.ExperimentalFactor.ontology_id)
ln.save(records)

❗ now recursing through parents: this only happens once, but is much slower than bulk saving


## Directly download a lamin database

In this context one ca either directly download a lamin database (here the cellxgene database as example).

In [ ]:
cx_dataset = ln.Dataset.using("laminlabs/cellxgene").one()
cx_dataset, len(cx_dataset.files.all())

In [ ]:
mydataset = utils.load_dataset_local(lb, cx_dataset, "~/scdataloader/", name="cellxgene-local", description="the full cellxgene database", only=(21,200))

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


❗ record with EtxwAoHUyGTdCB8swqaf already exists on default database: File(uid='EtxwAoHUyGTdCB8swqaf', key='cell-census/2023-07-25/h5ads/0738f538-ff2f-4346-b2eb-72704c291188.h5ad', suffix='.h5ad', accessor='AnnData', description='High Resolution Slide-seqV2 Spatial Transcriptomics Enables Discovery of Disease-Specific Cell Neighborhoods and Pathways', size=12110480, hash='8PORLeHJm9wYZ5MUc4AlSw-2', hash_type='md5-n', visibility=1, key_is_virtual=False, updated_at=2023-11-28 22:44:40 UTC, storage_id=2, transform_id=11, run_id=16, created_by_id=1)
File cell-census/2023-07-25/h5ads/0738f538-ff2f-4346-b2eb-72704c291188.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()
File cell-census/2023-07-25/h5ads/07428d73-fdea-4bd4-a801-94b00c4d961c.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/asyncio/sslproto.py:320: ResourceWarning: unclosed transport <asyncio.sslproto._SSLProtocolTransport object at 0x7f8b49dda980>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


File cell-census/2023-07-25/h5ads/07854d9c-5375-4a9b-ac34-fa919d3c3686.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/asyncio/sslproto.py:320: ResourceWarning: unclosed transport <asyncio.sslproto._SSLProtocolTransport object at 0x7f8b450b9420>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


...

File cell-census/2023-07-25/h5ads/0b4a15a7-4e9e-4555-9733-2423e5c66469.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/asyncio/sslproto.py:320: ResourceWarning: unclosed transport <asyncio.sslproto._SSLProtocolTransport object at 0x7f8b49b7a0e0>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


File cell-census/2023-07-25/h5ads/0b75c598-0893-4216-afe8-5414cab7739d.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/asyncio/sslproto.py:320: ResourceWarning: unclosed transport <asyncio.sslproto._SSLProtocolTransport object at 0x7f8b49dda980>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


File cell-census/2023-07-25/h5ads/0ba636a1-4754-4786-a8be-7ab3cf760fd6.h5ad already exists in storage



## Or Preprocessing + Download

In this case we use a custom made function that applies a preprocessing after downloading each files in the database.

In [ ]:
from scdataloader.preprocess import Preprocessor
import scanpy as sc
import numpy as np

In [ ]:
DESCRIPTION='preprocessed by scprint'

In [ ]:
# Here we also add some additional preprocessing (happens at the beginning of the preprocessing function) and post processing (happens at the end of the preprocessing function)
# this serves as an exemple of the flexibility of the function

def additional_preprocess(adata):
    adata.obs = adata.obs.replace({'self_reported_ethnicity_ontology_term_id':{
        'multiethnic':'unknown',
        'American':'unknown',
        'Jewish Israeli': 'unknown',
        'na':'unknown',
    }}) #multi ethnic will have to get renamed
    adata.obs['cell_culture'] = False
    # if cell_type contains the word "(cell culture)" then it is a cell culture and we mark it as so and remove this from the cell type
    loc = adata.obs['cell_type_ontology_term_id'].str.contains("(cell culture)")
    if loc.sum()>0:
        adata.obs.loc[loc, 'cell_culture'] = True
        adata.obs.loc[loc, 'cell_type_ontology_term_id'] = adata.obs.loc[loc, 'cell_type_ontology_term_id'].str.replace(" (cell culture)", "")
    return adata

def additional_postprocess(adata):
    # define the "up to" 10 neighbors for each cells and add to obs
    # compute neighbors
    # need to be connectivities and same labels [cell type, assay, dataset, disease]
    # define the "neighbor" up to 10(N) cells and add to obs
    # define the "next time point" up to 5(M) cells and add to obs  # step 1: filter genes
    sc.tl.diffmap(adata)
    # create a meta group
    adata.obs['dpt_group'] = adata.obs['leiden_1'].astype(str) + "_" + adata.obs['disease_ontology_term_id'].astype(str) + "_" + adata.obs['cell_type_ontology_term_id'].astype(str) + "_" + adata.obs['tissue_ontology_term_id'].astype(str) #+ "_" + adata.obs['dataset_id'].astype(str)

    # if group is too small
    okgroup = [i for i, j in adata.obs['dpt_group'].value_counts().items() if j>=10]
    not_okgroup = [i for i, j in adata.obs['dpt_group'].value_counts().items() if j<3]
    # set the group to empty
    adata.obs.loc[adata.obs['dpt_group'].isin(not_okgroup), 'dpt_group'] = ''
    adata.obs['heat_diff'] = np.nan
    # for each group
    for val in set(okgroup):
        if val == '':
            continue
        # get the best root cell
        eq = adata.obs.dpt_group==val
        loc = np.where(eq)[0]

        root_ixs = loc[adata.obsm["X_diffmap"][eq, 0].argmin()]
        adata.uns["iroot"] = root_ixs
        # compute the diffusion pseudo time from it
        sc.tl.dpt(adata)
        adata.obs.loc[eq, 'heat_diff'] = adata.obs.loc[eq, 'dpt_pseudotime']
        adata.obs.drop(columns=['dpt_pseudotime'], inplace=True)

    #sort so that the next time points are aligned for all groups
    adata = adata[adata.obs.sort_values(['dpt_group','heat_diff']).index]
    #to query N next time points we just get the N elements below and check they are in the group
    # to query the N nearest neighbors we just get the N elements above and N below and check they are in the group
    return adata

do_preprocess = Preprocessor(lb, additional_postprocess=additional_postprocess, additional_preprocess=additional_preprocess)


In [ ]:
preprocessed_dataset = do_preprocess(mydataset, start_at=11, description=DESCRIPTION)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()
0


AnnData object with n_obs × n_vars = 69709 × 25701
    obs: 'n_genes', 'sample', 'percent_mito', 'n_counts', 'batch', 'S_score', 'G2M_score', 'phase', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'is_doublet', 'lineage', 'dataset', 'lineageSomatic', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'gene_symbols', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'default_embedding', 'lineageSomatic_colors', 'lineage_colors', 'schema_version', 'title'
    obsm: 'X_scVI', 'X_umap'
Removed 49 genes.
Seeing 31596 outliers (45.33% of total dataset):
❗ no run & tra

... storing 'dpt_group' as categorical
... storing 'gene_symbols' as categorical
... storing 'symbol' as categorical
... storing 'ncbi_gene_id' as categorical
... storing 'biotype' as categorical
... storing 'description' as categorical
... storing 'synonyms' as categorical


1
AnnData object with n_obs × n_vars = 13623 × 59357
    obs: 'roi', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'Biotype', 'Chromosome', 'End', 'Gene', 'Start', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'batch_condition', 'schema_version', 'title'
    obsm: 'X_UMAP', 'X_tSNE'
Removed 128 genes.
Seeing 8587 outliers (63.03% of total dataset):
❗ no run & transform get linked, consider passing a `run` or cal

... storing 'dpt_group' as categorical
... storing 'symbol' as categorical
... storing 'ncbi_gene_id' as categorical
... storing 'biotype' as categorical
... storing 'description' as categorical
... storing 'synonyms' as categorical


...

7
AnnData object with n_obs × n_vars = 3083 × 16443
    obs: 'n_counts', 'n_genes', 'percent.mt', 'Adipocyte', 'Cardiomyocyte', 'Endothelial', 'Fibroblast', 'Lymphoid', 'Mast', 'Myeloid', 'Neuronal', 'Pericyte', 'Cycling.cells', 'vSMCs', 'cell_type_original', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'donor_id', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'X_approximate_distribution', 'default_embedding', 'schema_version', 'title'
    obsm: 'X_pca', 'X_spatial', 'X_umap'
Removed 37 genes.
Seeing 1481 outliers (48.04% of total dataset):
❗ no run & transform get linked, consider passing a `ru

... storing 'dpt_group' as categorical
... storing 'symbol' as categorical
... storing 'ncbi_gene_id' as categorical
... storing 'biotype' as categorical
... storing 'description' as categorical
... storing 'synonyms' as categorical


8
AnnData object with n_obs × n_vars = 11265 × 59357
    obs: 'roi', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'Biotype', 'Chromosome', 'End', 'Gene', 'Start', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'batch_condition', 'schema_version', 'title'
    obsm: 'X_UMAP', 'X_tSNE'
Removed 128 genes.
Seeing 6290 outliers (55.84% of total dataset):
❗ no run & transform get linked, consider passing a `run` or cal

... storing 'dpt_group' as categorical
... storing 'symbol' as categorical
... storing 'ncbi_gene_id' as categorical
... storing 'biotype' as categorical
... storing 'description' as categorical
... storing 'synonyms' as categorical


❗ no run & transform get linked, consider passing a `run` or calling ln.track()


In [56]:
#we have processed that many files
len(ln.File.filter(version='2', description=DESCRIPTION))

28

In [58]:
# we can load a preprocessed anndata like this
adata = ln.File.filter(version='2', description=DESCRIPTION)[23].backed()
adata.obs

In [ ]:
# I need to remake the dataset as it failed for some files and I had to restart at position 11 (As you can see in the preprocess() function)
name="preprocessed dataset"
description="preprocessed dataset using scprint"
dataset = ln.Dataset(ln.File.filter(version='2', description=DESCRIPTION), name=name, description=description)
dataset.save()
dataset.files.count()

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


AttributeError: 'Dataset' object has no attribute 'files'